In [ ]:
# Google Drive Access:

import io
import pandas as pd

# Import packages for connecting to Google Drive
from google.oauth2 import service_account
from googleapiclient.discovery import build
# from googleapiclient.errors import HttpError
from googleapiclient.http import MediaIoBaseDownload

# CHANGE PATH NAME to where the service account credential is on your computer.
credentials_path = '/Users/imogengardiner/Documents/exercise_credential.json'

# REPLACE FOLDER_ID with the folder in Google Drive you are interested in importing data from.
    # Instructions: Search online for the HIIT and Endurance Study shared google drive. Navigate to the folder with data you are interested in. 
    # Look at the web URL. Replace 'folder_id' with the end of the URL.
    # Example for getting Polar Data: HIIT and Endurance Study Drive > Data > data > workout folder. The workout folder URL is https://drive.google.com/drive/folders/1a9Bmg89_9m9BaYLsLw52PS_m07FQ7CtZ.
    # Notice how the end of the web URL is the folder_id.
folder_id = '1PjDaMLxn3QvYqICxCxK5PF2-yhh8sNoa'

# Authenticate with Google Drive using service account credentials
credentials = service_account.Credentials.from_service_account_file(credentials_path, scopes=['https://www.googleapis.com/auth/drive'])
drive_service = build('drive', 'v3', credentials=credentials)

# Get files
def get_file_ids_from_dir(parent_id):
    # drive_service = setup_drive()
    results = drive_service.files().list(
        corpora='drive',
        driveId='0AB6End4Uf7P-Uk9PVA', # if you search for the HIIT and Endurance Study shared google drive, this ID is taken from the end of the pathname: https://drive.google.com/drive/folders/0AB6End4Uf7P-Uk9PVA
        q=f"'{parent_id}' in parents",
        includeItemsFromAllDrives=True,
        supportsAllDrives=True
    ).execute()
    files = results.get('files', [])
    if not files:
        raise Exception(f"Folder {parent_id} has no files!")
    # id_name = [{x['name'] : x['id']} for x in files]
    id_name = {}
    for x in files:
        id_name[x['name']] = x['id']
    return id_name

def load_data_from_drive(_drive_service, file_id):
    request = _drive_service.files().get_media(fileId=file_id)
    io_buffer = io.BytesIO()
    downloader = MediaIoBaseDownload(io_buffer, request)
    done = False
    while done is False:
        status, done = downloader.next_chunk()
    io_buffer.seek(0)
    df = pd.read_csv(io_buffer, parse_dates=['_realtime','_time'])
    return df

# Get file names from folder
fileids = get_file_ids_from_dir(folder_id)
print(fileids)

dfsl_list = []
for fname, fid in fileids.items():
    dfsl = load_data_from_drive(drive_service, fid)
    dfsl_list.append(dfsl)

print(dfsl_list)

In [ ]:
# Transfer code from 007 analysis and loop through every file in the sl/Fitbit folder, and create plots for each participant:

dfs = {}

for file in dfsl_list:
    df = pd.read_csv(file)
    
    # Store the DataFrame in the dictionary with the file name as the key
    dfs[file[:3]] = df

# Drop unnecessary columns from the DataFrame
for i in range(len(dfsl_list)):
    columns_to_remove = ['device', 'start_date', 'end_date', 'start_time', 'end_time', '_time', 'myphd_id', 'target_hr_45', 'target_hr_55', 'target_hr_70', 'target_hr_90', 'ppt_id', 'moy_abbr', 'tag', 'datatype', 'sourcetype', 'dname',
                    'myphd_date_shift']
    dfsl_list[i] = dfsl_list[i].drop(columns=columns_to_remove)
    
dfsl_list = dfsl_list[dfsl_list['enrollment_status']=='Completed']
                         

print(dfsl_list.head())

# Adjust the values to reflect seconds (rather than decaseconds)
dfsl_list_004['value'] = dfsl_list_004['value']*10

# Rename value to seconds
dfsl_list_004 = dfsl_list_004.rename(columns={'value': 'seconds'})

print(dfsl_list_004.head())